## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np
# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Jacksonville,US,30.33,-81.66,63.00,58,1,11.41,clear sky
1,1,Ushuaia,AR,-54.80,-68.30,50.00,61,75,29.97,broken clouds
2,2,Arraial Do Cabo,BR,-22.97,-42.03,78.80,83,75,11.41,broken clouds
3,3,Bonavista,CA,48.65,-53.11,36.00,58,48,17.00,scattered clouds
4,4,Mana,GF,5.66,-53.78,87.55,65,65,13.85,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 50
What is the maximum temperature you would like for your trip? 70


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Jacksonville,US,30.33,-81.66,63.00,58,1,11.41,clear sky
1,1,Ushuaia,AR,-54.80,-68.30,50.00,61,75,29.97,broken clouds
5,5,Busselton,AU,-33.65,115.33,68.00,46,16,2.62,few clouds
9,9,Bredasdorp,ZA,-34.53,20.04,66.20,39,0,11.41,clear sky
12,12,Saint-Francois,FR,46.42,3.91,66.00,63,0,4.70,clear sky
13,13,Puerto Ayora,EC,-0.74,-90.35,69.01,95,96,4.00,overcast clouds
14,14,Tuatapere,NZ,-46.13,167.68,55.99,97,100,5.01,overcast clouds
16,16,Saint-Philippe,RE,-21.36,55.77,68.00,68,75,14.99,broken clouds
19,19,Hermanus,ZA,-34.42,19.23,66.00,50,0,5.99,clear sky
22,22,Punta Arenas,CL,-53.15,-70.92,53.60,66,75,35.57,broken clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                131
City                   131
Country                128
Lat                    131
Lng                    131
Max Temp               131
Humidity               131
Cloudiness             131
Wind Speed             131
Current Description    131
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)].dropna()
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Jacksonville,US,30.33,-81.66,63.00,58,1,11.41,clear sky
1,1,Ushuaia,AR,-54.80,-68.30,50.00,61,75,29.97,broken clouds
5,5,Busselton,AU,-33.65,115.33,68.00,46,16,2.62,few clouds
9,9,Bredasdorp,ZA,-34.53,20.04,66.20,39,0,11.41,clear sky
12,12,Saint-Francois,FR,46.42,3.91,66.00,63,0,4.70,clear sky
13,13,Puerto Ayora,EC,-0.74,-90.35,69.01,95,96,4.00,overcast clouds
14,14,Tuatapere,NZ,-46.13,167.68,55.99,97,100,5.01,overcast clouds
16,16,Saint-Philippe,RE,-21.36,55.77,68.00,68,75,14.99,broken clouds
19,19,Hermanus,ZA,-34.42,19.23,66.00,50,0,5.99,clear sky
22,22,Punta Arenas,CL,-53.15,-70.92,53.60,66,75,35.57,broken clouds


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Jacksonville,US,63.00,clear sky,30.33,-81.66,
1,Ushuaia,AR,50.00,broken clouds,-54.80,-68.30,
5,Busselton,AU,68.00,few clouds,-33.65,115.33,
9,Bredasdorp,ZA,66.20,clear sky,-34.53,20.04,
12,Saint-Francois,FR,66.00,clear sky,46.42,3.91,
13,Puerto Ayora,EC,69.01,overcast clouds,-0.74,-90.35,
14,Tuatapere,NZ,55.99,overcast clouds,-46.13,167.68,
16,Saint-Philippe,RE,68.00,broken clouds,-21.36,55.77,
19,Hermanus,ZA,66.00,clear sky,-34.42,19.23,
22,Punta Arenas,CL,53.60,broken clouds,-53.15,-70.92,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

print("Beginning Data Retrieval     ")
print("-----------------------------")

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except IndexError:
         print("Hotel not found... skipping.")
            
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")        

Beginning Data Retrieval     
-----------------------------
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
-----------------------------
Data Retrieval Complete      
-----------------------------


In [9]:
# 7. Drop the rows where there is no Hotel Name
hotel_df = hotel_df.replace('',np.nan)
hotel_df = hotel_df.dropna()
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Jacksonville,US,63.00,clear sky,30.33,-81.66,Hyatt Regency Jacksonville Riverfront
1,Ushuaia,AR,50.00,broken clouds,-54.80,-68.30,Albatros Hotel
5,Busselton,AU,68.00,few clouds,-33.65,115.33,Observatory Guest House
9,Bredasdorp,ZA,66.20,clear sky,-34.53,20.04,Bredasdorp Country Manor
12,Saint-Francois,FR,66.00,clear sky,46.42,3.91,Manoir Du Bonheur
...,...,...,...,...,...,...,...
516,Pacasmayo,PE,65.28,overcast clouds,-7.40,-79.57,Hotel Libertad
517,Fukue,JP,64.02,clear sky,32.69,128.84,Goto Guesthouse Utojuku & SA Car Rental
518,La Palma,US,59.00,clear sky,33.85,-118.05,La Quinta Inn & Suites by Wyndham Buena Park
519,Te Anau,NZ,54.50,light rain,-45.42,167.72,Kingsgate Hotel Te Anau


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_Vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl><dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp}  °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Call the figure to plot the data.
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))